# Analisi Game Experience Questionnaire (GEQ)

Questo notebook analizza i dati raccolti mediante il Game Experience Questionnaire (GEQ), calcolando i punteggi delle varie componenti secondo le linee guida ufficiali e creando visualizzazioni utili per l'inclusione nella tesi.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from pathlib import Path

# Configurazione del notebook
%matplotlib inline
sns.set(style="whitegrid")
plt.rcParams['figure.figsize'] = (12, 8)

## Definizione delle Componenti GEQ

Secondo le linee guida ufficiali, il GEQ è suddiviso in diverse componenti per ogni modulo. Qui sotto definiamo quali domande appartengono a ciascuna componente.

In [ ]:
# Definizione delle componenti di ogni modulo
GEQ_COMPONENTS = {
    # Core Module
    'Modulo Principale': {
        'Competenza': [2, 10, 15, 17, 21],
        'Immersione Sensoriale e Immaginativa': [3, 12, 18, 19, 27, 30],
        'Flusso': [5, 13, 25, 28, 31],
        'Tensione/Fastidio': [22, 24, 29],
        'Sfida': [11, 23, 26, 32, 33],
        'Affetto Negativo': [7, 8, 9, 16],
        'Affetto Positivo': [1, 4, 6, 14, 20]
    },
    # In-Game Module
    'Modulo In-game': {
        'Competenza': [35, 42],
        'Immersione Sensoriale e Immaginativa': [34, 37],
        'Flusso': [38, 43],
        'Tensione': [39, 41],
        'Sfida': [45, 46],
        'Affetto Negativo': [36, 40],
        'Affetto Positivo': [44, 47]
    },
    # Post-Game Module
    'Modulo Post-game': {
        'Esperienza Positiva': [48, 52, 54, 55, 59, 63],
        'Esperienza Negativa': [49, 51, 53, 58, 61, 62],
        'Stanchezza': [57, 60],
        'Ritorno alla Realtà': [50, 56, 64]
    }
}

## Caricamento dei Dati

Per prima cosa, carichiamo il file CSV dei risultati del GEQ per analizzarlo.

In [ ]:
# Modifica il percorso al tuo file CSV
data_path = "./geq_all_results.csv"

# Carica i dati
df = pd.read_csv(data_path)

# Visualizza le prime righe per controllare
df.head()

In [ ]:
# Controlla quanti partecipanti e risposte abbiamo
print(f"Numero totale di risposte: {len(df)}")
print(f"Numero di partecipanti unici: {df['player_id'].nunique()}")
print("\nTipi di sessione:")
print(df['type'].value_counts())
print("\nModuli compilati:")
print(df['module'].value_counts())

## Calcolo dei Punteggi delle Componenti

Ora calcoleremo i punteggi medi per ogni componente di ogni modulo, per ogni partecipante.

In [ ]:
def calculate_component_scores(df):
    """
    Calcola i punteggi delle componenti per ogni partecipante e modulo
    """
    # Crea un dataframe per i risultati
    result_df = pd.DataFrame()
    
    # Ottieni i partecipanti unici
    participants = df['player_id'].unique()
    
    for participant_id in participants:
        participant_data = df[df['player_id'] == participant_id]
        
        # Estrai il tipo di sessione (in_game o end_session)
        session_type = participant_data['type'].iloc[0]
        
        # Prepara un record per i risultati di questo partecipante
        participant_result = {
            'player_id': participant_id,
            'timestamp': participant_data['timestamp'].iloc[0],
            'type': session_type
        }
        
        # Determina quali moduli sono stati completati
        modules = participant_data['module'].unique()
        
        for module_name in modules:
            # Ottieni i dati di questo modulo
            module_data = participant_data[participant_data['module'] == module_name]
            
            # Crea un dizionario che mappa ID domanda -> risposta
            responses_dict = dict(zip(module_data['question_id'], module_data['response']))
            
            # Calcola i punteggi per ogni componente di questo modulo
            if module_name in GEQ_COMPONENTS:
                for component_name, question_ids in GEQ_COMPONENTS[module_name].items():
                    # Calcola il punteggio medio per questa componente
                    component_scores = [responses_dict.get(q_id, np.nan) for q_id in question_ids]
                    valid_scores = [s for s in component_scores if not np.isnan(s)]
                    
                    if valid_scores:
                        component_score = sum(valid_scores) / len(valid_scores)
                    else:
                        component_score = np.nan
                    
                    # Aggiungi il punteggio al record del partecipante
                    column_name = f"{module_name}_{component_name}"
                    participant_result[column_name] = component_score
        
        # Aggiungi il record di questo partecipante al dataframe dei risultati
        result_df = pd.concat([result_df, pd.DataFrame([participant_result])], ignore_index=True)
    
    return result_df

# Calcola i punteggi delle componenti
component_scores = calculate_component_scores(df)

# Visualizza i risultati
component_scores.head()

## Analisi dell'Esperienza Complessiva

Ora che abbiamo calcolato i punteggi delle componenti, possiamo analizzare l'esperienza complessiva dei partecipanti.

In [ ]:
def analyze_overall_experience(component_df):
    """
    Analizza l'esperienza complessiva basata sui punteggi delle componenti
    """
    # Separa i dati per tipo di sessione
    in_game_data = component_df[component_df['type'] == 'in_game']
    end_session_data = component_df[component_df['type'] == 'end_session']
    
    results = {}
    
    if not in_game_data.empty:
        # Analisi In-Game
        in_game_components = [col for col in in_game_data.columns if col.startswith('In-game Module_')]
        
        in_game_avg = {}
        for component in in_game_components:
            clean_name = component.replace('In-game Module_', '')
            in_game_avg[clean_name] = in_game_data[component].mean()
        
        results['in_game'] = in_game_avg
    
    if not end_session_data.empty:
        # Analisi Core Module
        core_components = [col for col in end_session_data.columns if col.startswith('Core Module_')]
        
        core_avg = {}
        for component in core_components:
            clean_name = component.replace('Core Module_', '')
            core_avg[clean_name] = end_session_data[component].mean()
        
        results['core'] = core_avg
        
        # Analisi Post-Game Module
        post_components = [col for col in end_session_data.columns if col.startswith('Post-game Module_')]
        
        post_avg = {}
        for component in post_components:
            clean_name = component.replace('Post-game Module_', '')
            post_avg[clean_name] = end_session_data[component].mean()
        
        results['post_game'] = post_avg
    
    return results

# Analizza l'esperienza complessiva
analysis_results = analyze_overall_experience(component_scores)

# Visualizza i risultati
for module_key, components in analysis_results.items():
    print(f"\n{module_key.upper()} Module:")
    for component, score in components.items():
        print(f"{component}: {score:.2f}")

## Visualizzazioni dei Risultati

Ora creiamo alcune visualizzazioni dei risultati per aiutare l'interpretazione.

In [ ]:
def create_module_barplot(module_key, components, title):
    """
    Crea un grafico a barre per le componenti di un modulo
    """
    # Converti in un DataFrame per la visualizzazione
    df = pd.DataFrame(list(components.items()), columns=['Component', 'Score'])
    
    # Ordina per punteggio decrescente
    df = df.sort_values('Score', ascending=False)
    
    # Crea il grafico a barre
    plt.figure(figsize=(12, 6))
    ax = sns.barplot(x='Component', y='Score', data=df, palette='viridis')
    
    # Aggiungi etichette e titolo
    plt.title(title, fontsize=16)
    plt.xlabel('Component', fontsize=14)
    plt.ylabel('Average Score (0-4)', fontsize=14)
    plt.ylim(0, 4)  # Scala da 0 a 4 per tutti i grafici
    
    # Ruota le etichette delle componenti per leggibilità
    plt.xticks(rotation=45, ha='right')
    
    # Aggiungi i valori sopra ogni barra
    for i, p in enumerate(ax.patches):
        ax.annotate(f'{p.get_height():.2f}', 
                    (p.get_x() + p.get_width() / 2., p.get_height()), 
                    ha='center', va='bottom', fontsize=12, rotation=0)
    
    plt.tight_layout()
    
    return plt.gcf()

# Crea i grafici per ogni modulo
if 'core' in analysis_results:
    core_fig = create_module_barplot('core', analysis_results['core'], 'GEQ Core Module Component Scores')
    display(core_fig)
    
if 'in_game' in analysis_results:
    ingame_fig = create_module_barplot('in_game', analysis_results['in_game'], 'GEQ In-Game Module Component Scores')
    display(ingame_fig)
    
if 'post_game' in analysis_results:
    postgame_fig = create_module_barplot('post_game', analysis_results['post_game'], 'GEQ Post-Game Module Component Scores')
    display(postgame_fig)

## Confronto tra Esperienze Positive e Negative

Confrontiamo le esperienze positive e negative tra i moduli Core e Post-Game.

In [ ]:
# Crea un grafico di confronto per le componenti positive/negative
if 'core' in analysis_results and 'post_game' in analysis_results:
    positive_scores = [
        analysis_results['core'].get('Positive affect', 0), 
        analysis_results['post_game'].get('Positive Experience', 0)
    ]
    negative_scores = [
        analysis_results['core'].get('Negative affect', 0),
        analysis_results['post_game'].get('Negative experience', 0)
    ]
    
    labels = ['Core Module', 'Post-Game Module']
    
    # Crea il grafico a barre affiancate
    plt.figure(figsize=(10, 6))
    x = np.arange(len(labels))
    width = 0.35
    
    fig, ax = plt.subplots(figsize=(10, 6))
    rects1 = ax.bar(x - width/2, positive_scores, width, label='Positive Experience', color='green')
    rects2 = ax.bar(x + width/2, negative_scores, width, label='Negative Experience', color='red')
    
    ax.set_ylabel('Average Score (0-4)', fontsize=14)
    ax.set_title('Positive vs Negative Experience Comparison', fontsize=16)
    ax.set_xticks(x)
    ax.set_xticklabels(labels)
    ax.legend()
    ax.set_ylim(0, 4)
    
    # Aggiungi i valori sopra ogni barra
    for rect in rects1 + rects2:
        height = rect.get_height()
        ax.annotate(f'{height:.2f}',
                    xy=(rect.get_x() + rect.get_width() / 2, height),
                    xytext=(0, 3),  # 3 punti sopra la barra
                    textcoords="offset points",
                    ha='center', va='bottom')
    
    plt.tight_layout()
    display(plt.gcf())

## Grafico Radar delle Componenti del Core Module

Un grafico radar può essere utile per visualizzare il profilo complessivo dell'esperienza di gioco.

In [ ]:
# Crea un grafico radar per il Core Module
if 'core' in analysis_results:
    core_components = analysis_results['core']
    
    # Prepara i dati per il grafico radar
    categories = list(core_components.keys())
    values = list(core_components.values())
    
    # Chiudi il grafico aggiungendo il primo valore alla fine
    categories.append(categories[0])
    values.append(values[0])
    
    # Angoli per il grafico radar
    angles = np.linspace(0, 2*np.pi, len(categories), endpoint=False).tolist()
    angles += angles[:1]  # Chiudi il ciclo
    
    # Crea il grafico radar
    fig, ax = plt.subplots(figsize=(10, 10), subplot_kw=dict(polar=True))
    
    # Aggiungi gli assi
    plt.xticks(angles[:-1], categories[:-1], size=12)
    
    # Imposta i limiti dell'asse y
    ax.set_ylim(0, 4)
    
    # Disegna la linea del contorno
    ax.plot(angles, values, 'o-', linewidth=2, label='Core Module')
    ax.fill(angles, values, alpha=0.25)
    
    plt.title('GEQ Core Module Component Profile', size=20, y=1.1)
    
    plt.tight_layout()
    display(plt.gcf())

## Analisi per Partecipante

Se abbiamo dati di più partecipanti, possiamo confrontare le loro esperienze.

In [ ]:
# Seleziona solo le colonne delle componenti
component_columns = [col for col in component_scores.columns 
                    if col.startswith('Core Module_') or
                       col.startswith('In-game Module_') or
                       col.startswith('Post-game Module_')]

# Se abbiamo più di un partecipante
if len(component_scores) > 1:
    # Statistiche descrittive
    stats = component_scores[component_columns].describe()
    display(stats)
    
    # Per ogni partecipante, mostra le componenti con punteggio più alto e più basso
    for index, row in component_scores.iterrows():
        print(f"\nPartecipante: {row['player_id']}")
        
        # Seleziona i punteggi delle componenti per questo partecipante
        participant_scores = row[component_columns].dropna()
        
        if len(participant_scores) > 0:
            max_component = participant_scores.idxmax()
            min_component = participant_scores.idxmin()
            
            print(f"Componente con punteggio più alto: {max_component} ({participant_scores[max_component]:.2f})")
            print(f"Componente con punteggio più basso: {min_component} ({participant_scores[min_component]:.2f})")
            
    # Grafico per confrontare i partecipanti
    if len(component_scores) <= 10:  # Limita il grafico a 10 partecipanti max per leggibilità
        plt.figure(figsize=(15, 8))
        
        for col in component_columns:
            if component_scores[col].notna().sum() > 0:  # Crea un grafico solo se ci sono dati
                ax = sns.barplot(x='player_id', y=col, data=component_scores)
                plt.title(f'Confronto dei punteggi per {col}')
                plt.xticks(rotation=45)
                plt.ylim(0, 4)
                plt.tight_layout()
                display(plt.gcf())
                plt.close()

## Esportazione dei Risultati

Esportiamo i risultati finali per poterli utilizzare nella tesi.

In [ ]:
# Crea una directory per i risultati se non esiste
output_dir = Path("./geq_results")
output_dir.mkdir(exist_ok=True, parents=True)

# Salva i punteggi delle componenti in un file CSV
component_scores.to_csv(output_dir / "geq_component_scores.csv", index=False)
print(f"Punteggi delle componenti salvati in {output_dir / 'geq_component_scores.csv'}")

# Salva i risultati dell'analisi in un file JSON
import json
with open(output_dir / "geq_analysis_results.json", 'w') as f:
    # Converti valori numpy in float per la serializzazione JSON
    results_json = {}
    for module, components in analysis_results.items():
        results_json[module] = {comp: float(score) for comp, score in components.items()}
    
    json.dump(results_json, f, indent=4)
print(f"Risultati dell'analisi salvati in {output_dir / 'geq_analysis_results.json'}")

# Salva i grafici
if 'core_fig' in locals():
    core_fig.savefig(output_dir / "geq_core_components.png", dpi=300, bbox_inches='tight')
    print(f"Grafico del Core Module salvato in {output_dir / 'geq_core_components.png'}")
    
if 'ingame_fig' in locals():
    ingame_fig.savefig(output_dir / "geq_ingame_components.png", dpi=300, bbox_inches='tight')
    print(f"Grafico dell'In-Game Module salvato in {output_dir / 'geq_ingame_components.png'}")
    
if 'postgame_fig' in locals():
    postgame_fig.savefig(output_dir / "geq_postgame_components.png", dpi=300, bbox_inches='tight')
    print(f"Grafico del Post-Game Module salvato in {output_dir / 'geq_postgame_components.png'}")

## Conclusioni

Con questa analisi hai ottenuto punteggi per ogni componente del GEQ, visualizzazioni utili per interpretare i risultati e file esportati che puoi utilizzare nella tua tesi sui roguelike.

Puoi utilizzare queste informazioni per:

1. Valutare l'esperienza complessiva di gioco (Core Module)
2. Comprendere l'esperienza durante il gioco (In-Game Module)
3. Analizzare l'esperienza dopo il gioco (Post-Game Module)
4. Confrontare diverse versioni del tuo algoritmo di generazione procedurale
5. Identificare punti di forza e aree di miglioramento

Queste metriche ti aiuteranno a dimostrare scientificamente il valore del tuo algoritmo di generazione procedurale per i roguelike, rendendo la tua tesi più solida e basata su dati concreti.